# Experiment: Bag of Tokens Neural Network

This experiment seeks to create a neural network to predict the next token in a sequence, given a bag of tokens representation of the preceeding tokens

In [1]:
import random

from collections.abc import Iterable
from typing import Union

import pandas as pd
import numpy as np
import torch

from torch import nn

## Initialize experiment parameters

In [2]:
random.seed(42)

train_size = 0.8

## Load the training data

In [3]:
(dataset := pd.read_csv('../data/text/text_emotion.csv')).head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


## Preprocess the data

In [4]:
# Convert the dataset to lowercase.
dataset = dataset['content'].str.lower()
# Split the dataset into train and test sets.
train_dataset, test_dataset = dataset[: (train_dataset_size := int(dataset.shape[0] * train_size))], dataset[train_dataset_size:].reset_index(drop=True)

## Transform the data

In [5]:
class Tokenizer:
    end_token = "<E>"

    @staticmethod
    def tokenize(document: str, is_complete: str = True) -> list[str]:
        tokens = [Tokenizer.end_token] + list(document) 
        if is_complete and tokens[len(tokens) - 1] != Tokenizer.end_token:
            tokens.append(Tokenizer.end_token)
        return tokens

In [6]:
class Codec:
    end_token = "<E>"
    
    def __init__(self, documents: Iterable[str]): 
        self.vocab = set(Tokenizer.end_token)
        for document in documents:
            for token in Tokenizer.tokenize(document):
                self.vocab.add(token)
        self.vocab_size = len(self.vocab)
        self.encoding_by_token = {token: encoding for encoding, token in enumerate(self.vocab)}
        self.token_by_encoding = {encoding: token for token, encoding in self.encoding_by_token.items()} 

    def encode(self, tokens: Union[str, list[str], tuple[str, ...]]) -> Union[str, list[str], tuple[str, ...]]:
        if isinstance(tokens, str):
            encodings = self.encoding_by_token[tokens]
        elif (isinstance(tokens, list) or isinstance(tokens, tuple)) and all(isinstance(token, str) for token in tokens):
            encodings = type(tokens)((self.encoding_by_token[token] for token in tokens))
        return encodings

    def decode(self, encodings: Union[int, list[int], tuple[int, ...]]) -> Union[int, list[int], tuple[int, ...]]:
        if isinstance(encodings, int):
            tokens = self.token_by_encoding[encodings]
        elif (isinstance(encodings, list) or isinstance(encodings, tuple)) and all(isinstance(encoding, int) for encoding in encodings):
            tokens = type(encodings)((self.token_by_encoding[encoding] for encoding in encodings))
        return tokens

In [7]:
class ContextTokenPair:
    def __init__(self, context, token, document_index=None, context_start_index=None, context_size=None):
        self.context = context
        self.token = token
        self.document_index = document_index
        self.context_index = (context_start_index, context_start_index + context_size)
        self.token_index = context_start_index + context_size

    def __str__(self):
        return "ContextTokenPair(context={}, token='{}', document_index={}, context_index={}, token_index={})".format(
            self.context, self.token, self.document_index, self.context_index, self.token_index
        )
    
    def __repr__(self):
        return str(self)
    

def to_context_token_pairs(documents: Iterable[str], context_size: int) -> Iterable[ContextTokenPair]:
    """Generates context-token pairs from the given documents.
    
    Context-token pairs are generated in the order they appear in the documents with context length fixed to the given context size.
    """
    for document_index, document in enumerate(documents):
        tokens = Tokenizer.tokenize(document)
        # Skip documents lacking sufficient tokens for a context-token pair with the required size.
        if (total_tokens := len(tokens)) < context_size + 1:
            continue
        for context_start_index in range(total_tokens - context_size):
            context = tokens[context_start_index : context_start_index + context_size]
            token = tokens[context_start_index + context_size]
            yield ContextTokenPair(context, token, document_index=document_index, context_start_index=context_start_index, context_size=context_size)


In [8]:
class TrainingBatch:
    def __init__(self, examples: Iterable[ContextTokenPair]):
        self.examples = examples
        self.batch_size = len(examples)

    def encode(self, codec: Codec):
        X, y = torch.zeros((self.batch_size, codec.vocab_size)), torch.zeros((self.batch_size, codec.vocab_size))
        for i, example in enumerate(self.examples):
            for encoding in codec.encode(example.context):
                X[i][encoding] += 1
            y[i][codec.encode(example.token)] = 1
        return X, y
    
    def __str__(self):
        return "TrainingBatch(batch_size={}, batch=[{}])".format(
            self.batch_size, "\n" + "".join(["  " + example.__str__() + "\n" for example in self.examples])
        )
    
    def __repr__(self):
        return self.__str__()
    

def to_training_batches(training_examples: Iterable[ContextTokenPair], batch_size: int) -> Iterable[TrainingBatch]:
    batch = [None] * batch_size
    for i, training_example in enumerate(training_examples):
        batch[(curr_index := i % batch_size)] = training_example
        if curr_index == batch_size - 1:
            yield TrainingBatch(batch)

In [9]:
def get_random_training_example(dataset, context_size) -> ContextTokenPair:
    """Returns a random training example from the dataset."""
    # Select documents at random until one with the minimum number of tokens is found.
    while True:
        document = dataset[(document_index := random.randrange(0, dataset.shape[0]))]
        if len(tokens := Tokenizer.tokenize(document)) >= context_size + 1:
            break
    
    # Select a random context from the document.
    context_index = random.randrange(0, len(tokens) - context_size)
    token_index = context_index + context_size
    context, token = tokens[context_index: token_index], tokens[token_index]
    return ContextTokenPair(context, token, document_index, context_index, token_index)

## Create the training batches

In [19]:
codec = Codec(train_dataset)
training_examples = to_context_token_pairs(train_dataset, context_size=7)
training_batches = to_training_batches(training_examples, batch_size=8)

## Define the model

For the model we will use a simple neural network, 5 layers deep as a test. Once the learning ability of this model is proven we can evaulate using a larger neural network

In [20]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(codec.vocab_size, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            # nn.Linear(32, 32),
            # nn.ReLU(),
            # nn.Linear(32, 32),
            # nn.ReLU(),
            # nn.Linear(32, 32),
            # nn.ReLU(),
            # nn.Linear(32, 32),
            # nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, codec.vocab_size)
        )
    
    def forward(self, x):
        return self.layers(x)

## Train the model

In [ ]:
ngram_size = 3
batch_size = 16
num_train_batches = 40_000
num_test_batches = 10_000
num_epochs = 128
print_interval = 5_000

model = Model().to(device)
loss_fn = nn.CrossEntropyLoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
for epoch_no in range(1, num_epochs+1):
    print("-------------------------------------------")
    print(f"Epoch #{epoch_no}")
    # train 
    total_loss = 0
    total_correct = 0
    for batch_no in range(1, num_train_batches+1):
        inputs, targets = sample_batch_v2(train_dataset, batch_size=batch_size, ngram_size=ngram_size, device=device)
        logits = model(inputs)
        predictions = logits.argmax(dim=1)
        loss = loss_fn(logits, targets)
        loss.backward(); optimizer.step(); optimizer.zero_grad()
        total_loss += loss
        correct = (predictions == targets).type(torch.float).sum().item()
        total_correct += correct
        # print the batch and average stats
        if batch_no % print_interval == 0:
            print(f"  [{batch_no} / {num_train_batches}]: loss={(loss / batch_size):>7f}, Accuracy={((correct / batch_size) * 100):>3f}%, Avergage loss={(total_loss / (batch_size * batch_no)):>7f}, Accuracy={((total_correct / (batch_size * batch_no))* 100):>3f}%")
            # print(f"  [{batch_no} / {num_train_batches}]: ")
    
    # evaluate
    test_correct = 0
    test_loss = 0
    for batch_no in range(num_test_batches):
        X, y = sample_batch_v2(test_dataset, batch_size=batch_size, ngram_size=ngram_size, device=device)
        with torch.no_grad():
            y_pred = model(X)
            test_loss += loss_fn(y_pred, y).item()
            test_correct += (y_pred.argmax(1) == y).type(torch.float).sum().item()
    print(f"  Evaluation -- Test Accuracy: {((test_correct / (num_test_batches * batch_size)) * 100):>3f}%, Avg loss[{test_loss / (num_test_batches * batch_size)}]")
    print()
    

### Make predictions

In [ ]:
def predict_next_token_proba(ngram, top_k=5):
    ngram = Codec.encode_all(ngram)
    with torch.no_grad():
        encoded_ngram = to_bag_of_words(ngram, size=vocab_size).reshape(1, -1).to(device)
        logits = model(encoded_ngram)
        proba = nn.Softmax(dim=1)(logits)[0]
        if top_k == -1:
            top_k = proba.shape[0]
        top_predictions = proba.topk(top_k).indices.tolist()
    return [(f"#{k+1}", Codec.decode(prediction), proba[prediction]) for k, prediction in enumerate(top_predictions)]

In [ ]:
def predict_next_token(ngram, method='sample'):
    ngram = Codec.encode_all(ngram)
    with torch.no_grad():
        encoded_ngram = to_bag_of_words(ngram, size=vocab_size).reshape(1, -1).to(device)
        logits = model(encoded_ngram)
        proba = nn.Softmax(dim=1)(logits)[0]
        if method == 'sample':
            next_token = torch.multinomial(proba, num_samples=1)[0].item()
        elif method == 'top':
            next_token = proba.topk(1).indices.tolist()[0]
    return (next_token, Codec.decode(next_token))

In [ ]:
def get_last_ngram(tokens: list[str], ngram_size: int) -> list[str]:
    return tokens[max(0, len(tokens) - ngram_size): max(0, len(tokens) - ngram_size) + ngram_size]

def to_bag_of_words(encoded_tokens, size: int) -> torch.Tensor:
    bow = torch.zeros(size)
    for encoded_token in encoded_tokens:
        bow[encoded_token] += 1
    return bow

def get_random_token(encoded: bool = False):
    random_token_id = random.randint(0, vocab_size)
    if encoded:
        return random_token_id
    return Codec.decode(random_token_id)

def complete_text(prompt: str = None, max_tokens = 80):
    tokens = Codec.encode_all(Tokenizer.tokenize(prompt, is_complete=False))
    is_text_size_limit_reached = lambda: len(tokens) >= max_tokens
    is_last_token_end_token = lambda: tokens[len(tokens) - 1] == Codec.encode(END_TOKEN) and len(tokens) != 1
    is_text_complete = lambda: is_text_size_limit_reached() or is_last_token_end_token()
    while not is_text_complete():
        last_ngram = get_last_ngram(tokens, ngram_size=3)
        next_token, _ = predict_next_token(ngram, method='sample')
        tokens.append(next_token)
    return len(tokens), "".join(Codec.decode_all(tokens[1: -1]))

In [ ]:
complete_text('dog')

In [ ]:
predict_next_token_proba('g e')

In [ ]:
get_similar_next_token_counts('g e').loc['g e'].sum_values()

## Analysis & Review

### Why doesn't this work?

Let's break down the dataset into it's constituent ngrams

In [ ]:
unique_ngrams = set()
for document in train_dataset:
    tokens = Tokenizer.tokenize(document)
    for ngram in to_ngrams(tokens, ngram_size=ngram_size):
        unique_ngrams.add(tuple(ngram))
        
print(f"There are {vocab_size} unique tokens and {len(unique_ngrams)} unique ngrams in the dataset")

In [ ]:
id_by_ngram = {ngram: idx for idx, ngram in enumerate(unique_ngrams)}
ngram_by_id = {idx: ngram for ngram, idx in id_by_ngram.items()}

class NgramCodec:
    @staticmethod
    def encode(ngram: str) -> int:
        if isinstance(ngram, str) or isinstance(ngram, list):
            ngram = tuple(ngram)
        return id_by_ngram[ngram]

    @staticmethod
    def decode(encoded_ngram: int) -> str:
        return ngram_by_id[encoded_ngram]

In [ ]:
next_token_counts = np.zeros((len(unique_ngrams), vocab_size))
for document in train_dataset:
    tokens = Tokenizer.tokenize(document)
    for context, token in to_context_token_pairs(tokens, ngram_size=ngram_size):
        next_token_counts[NgramCodec.encode(context), Codec.encode(token)] += 1
next_token_counts = pd.DataFrame(next_token_counts, index=["".join(ngram) for ngram in ngram_by_id.values()], columns=list(token_by_id.values()))
next_token_counts

In [ ]:
# clone next token counts as the base for next token proba
next_token_proba = torch.zeros((len(unique_ngrams), vocab_size))
        
next_token_proba = next_token_proba / next_token_proba.sum(dim=1, keepdim=True)
next_token_proba_df = pd.DataFrame(next_token_proba.numpy(), 
                                   columns=list(token_by_id.values()),
                                   index=["".join(ngram) for idx, ngram in ngram_by_id.items()]) 
next_token_proba_df

#### Let's Understand With An Example

First let's see the model predictions for the trigram 'cab'

In [ ]:
call_model(('c', 'a', 'b'))

Note that the probabilities for each token aren't really that far apart

Given that our model accepts a bag of tokens, our model does not learn relationships of ordered tokens. Whether we pass 'cab' or 'bac' to the model all it will see are three 1s at position 23, 76 and 81.

This means that all ngrams with combinations of these three characters will factor into the prediction about the next character, which will inherently produce some ambiguity in the prediction. i.e if the model sees 'cab' we would like to assume that 'i' may be prediction for a word like cabin, but considering that the model will see the same input for the ngram 'bac' in which case we may want 'k' to be the prediction to form back, the model will now have to balance both of these in the prediction based on their frequency of occuring.

#### Let's see an example

In [ ]:
Codec.decode_all(call_model(['e', 'l', 'l']))

'aab', 'baa' = 0
'aab', 'aca' = 1  // {'a': 2, 'b': 1} {'a': 2, 'c': 1}
'aab', 'acc' = 2



In [ ]:
def is_similar(ngram, target):
    return similarity(target, ngram) > 0

def get_similar_ngrams(target, threshold=0):
    similar_ngrams = [(s, n) for n in unique_ngrams if (s := similarity(target, n)) >= threshold] # get all similar ngrams with their similarity score
    similar_ngrams.sort(key=lambda n: n[0], reverse=True) # sort the ngrams by their similarity score
    return [ngram[1] for ngram in similar_ngrams] 

In [ ]:
def get_similar_next_token_counts(ngram, threshold=0):
    similar_ngrams = get_similar_ngrams(ngram, threshold=threshold)
    return next_token_counts.loc[["".join(ngram) for ngram in similar_ngrams]]

In [ ]:
call_model((' ', ' ', 'x'))

In [ ]:
get_similar_next_token_counts('  x', threshold=1.0)

In [ ]:
get_similar_next_token_counts(('  '), threshold=0.6).sum().sort_values(ascending=False)[:20]

In [ ]:
next_token_proba_df.loc['  x'].sort_values(ascending=False)

In [ ]:
get_similar_next_token_counts(('x'))['<E>'].sort_values(ascending=False)

In [ ]:
from collections import Counter

In [ ]:
a = Counter({'a': 1, 'b': 3})
b = Counter({'b': 2, 'c': 1})
What i want is the difference 
a - b

In [ ]:
def distance(from_, to_):
    from_ = Counter(from_)
    to_ = Counter(to_)
    distance = 0
    for token, _ in from_.items():
        distance += abs(from_[token] - to_[token])
    return distance

def similarity(a, b):
    """Calculates the similarity between two ngrams"""
    num_tokens = len(a)
    d = distance(a, b)
    return round((num_tokens - d) / num_tokens, 4)

In [ ]:
def get_similar_ngram_next_token_probas(target):
    similar_ngrams = [(s, n) for n in unique_ngrams if (s := similarity((target), n)) > 0] # get all similar ngrams with their similarity score
    similar_ngrams.sort(key=lambda n: n[0], reverse=True) # sort the ngrams by their similarity score
    return next_token_proba_df.loc[["".join(ngram[1]) for ngram in similar_ngrams]] # return the next token probas for similar ngrams

In [ ]:
get_similar_ngram_next_token_probas(('c', 'a', 'r'))#.loc['<E>c\''].sort_values(ascending=False)

In [ ]:
find_documents_containing(train_dataset, "c'm")

In [ ]:
def calculate_ngram_distance(from_, to):
    counts = Counter(from_)

In [ ]:
def get_similar_ngram_probas(target_ngram):
    counts_by_token = Counter(ngram)
    for ngram in unique_ngrams:
        if distnace := get_distance(target_ngram, ngram) is not 0:
            distances[distance].add(ngram):
    

In [ ]:
call_model(('car'))

In [ ]:
[ngram for ngram in unique_ngrams if ('e' in ngram and 'l' in ngram)]

In [ ]:
def find_documents_containing(documents, substr):
    matching_documents = []
    for document in documents:
        if substr in document:
            matching_documents.append(document)
    return matching_documents

In [ ]:
find_documents_containing(train_dataset, 'le8')

In [ ]:
Codec.decode(1)

In [ ]:
next_token_proba[ngram

In [ ]:
a = [1, 2, 3]
to_ngrams(a, ngram_size=1)

In [ ]:
a = torch.zeros(5,5).random_(5)
a

In [ ]:
a_sum = a.sum(dim=1, keepdim=True)
a_sum

In [ ]:
a / a_sum

In [ ]:
tuples = set()

In [ ]:
t1 = ('a', 'b', 'c')

In [ ]:
tuples.add(t1)

In [ ]:
pred = pred.argmax(dim=1)
pred

In [ ]:
targets = torch.empty(3, dtype=torch.long).random_(3)
targets

In [ ]:
nn.functional.cross_entropy(logits, targets)

In [ ]:
a = torch.tensor([2., 2., 5.])
b = torch.tensor([3., 2., 5.])

In [ ]:
(a == b).sum()

In [ ]:
def generate_text(prompt: str = None, max_tokens=280):
    if prompt is None:
        encoded_tokens = [Codec.encode(END_TOKEN)]
    
    # TODO: Improve this condition.
    while (len(encoded_tokens) == 1 or encoded_tokens[len(encoded_tokens) - 1] != Codec.encode(END_TOKEN)) and len(encoded_tokens) < max_tokens:
        last_ngram = get_last_ngram(encoded_tokens, ngram_size=3)
        with torch.no_grad():
            # print(f"Current ngram: {last_ngram}")
            # print(f"Current ngram as bow: {to_bag_of_words(Codec.encode_all(last_ngram), size=vocab_size)}")
            encoded_ngram = to_bag_of_words(last_ngram, size=vocab_size).reshape(1, -1).to(device)
            # print(encoded_ngram)
            # break
            logits = model(encoded_ngram)
            proba = nn.Softmax(dim=1)(logits)[0]
            top_predictions = proba.topk(5).indices.tolist()
            # print(top_predictions)
            # break
            # print("Top 5 most likely next tokens:")
            # for i, encoding in enumerate(top_predictions):
            #     print(f'    #{i}. "{Codec.decode(encoding)}"')
            # break
            next_token = torch.multinomial(proba, num_samples=1)[0].item()
            # print(f'"{Codec.decode(next_token)}" was chosen')
            encoded_tokens.append(next_token)
    return len(encoded_tokens), "".join(Codec.decode_all(encoded_tokens[1: -1]))

In [ ]:
for i in range(1):
    print(f"#{i+1}: {generate_text()[1]}")

#### Possible reasons for poor performance:

#### Loss of ordering due to bag of words representation
The current implementation takes an ngram, an inherently ordered structure and destroy the order by representing it as a bag of words / bag of tokens. The algorithm them is expected to 

#### Sparsity of the input data
Input is the size of the vocabulary which in our case is ~100 characters. Since the ngram size used for experiment was often less than 5, at any given time only 5 of ~100 input values would be non-zero. This could have made learning difficult (investigate this)

#### Type of neural network used / Depth & Shape of the neural network
The neural network chosen does not have an inherrent notion of ordering and works with data at a specific point in time. This combined with the loss of ordering information in the input may result in poor performance. There is also an admitted lack of experience with regards to optimizing the shape of a neural network for next character prediction. There may be a way to optimize just a regular MLP for this task but the experience is lacking.

What can I learn coming out of this?

1. Why does sparsity affect the learning ability of a neural network?
2. How does learning rate affect the learning ability of neural network? What happens when learning rate values become too large or small?
3. How does a neural network learn in the first place? Can you learn to guestimate how well a given neural network will do for a specific dataset / task
4. How does cross entropy loss work? Why does the formula use the exponent rather than simple absolution.

This current implementation learns to predict the most common character to follow an ngram containing any permutation of tokens in the bag.

An effective way to confirm this would be to pick the tokens of the most common ngram in the dataset, then pull out all the permutations of that token and print their next token probabilities.

Compare this then to the token probabilities calculated by the neural network. The probabilities should almost look like joint probabilities of the token permutations / ngrams in the dataset

What about a 1-d convolutional neural network?